In [1]:
EXPERIMENT_NAME = "mlflow_tutorial"
EXPORT_MODEL_PATH = "toy_model.pkl"
RANDOM_STATE = 0

In [2]:
import mlflow
from mlflow.tracking import MlflowClient

import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pickle

## Load toy data and prep

In [3]:
# Load toy data 
features, classes = load_boston(return_X_y=True)

In [4]:
# Split train and test data 
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    classes, 
                                                    test_size=0.3, 
                                                    random_state=RANDOM_STATE)

## Train and export toy model

In [5]:
# Sample parameter
PCA_VAR = 1

In [6]:
# Set pipeline  
regressor = make_pipeline(StandardScaler(), 
                          PCA(n_components=PCA_VAR, random_state=RANDOM_STATE), 
                          LinearRegression())
# ... and train
regressor.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=1, random_state=0)),
                ('linearregression', LinearRegression())])

In [7]:
# Predict and calculate MSE
test_prediction  = regressor.predict(X_test)
mse_test = mean_squared_error(y_test, test_prediction)

In [8]:
# Export model
with open(EXPORT_MODEL_PATH, 'wb') as handle:
    pickle.dump(regressor, handle)

## Track with Mlflow

In [9]:
# Initialize client and experiment
client = MlflowClient()
mlflow.set_experiment(EXPERIMENT_NAME)
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

In [10]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id):
    mlflow.log_param("pca_var", PCA_VAR)      # Track model parameter
    mlflow.log_metric("MSE", mse_test)        # Track error value
    mlflow.log_artifact(EXPORT_MODEL_PATH)    # Track exported model